## Descrição

Notebook exploratorio do solver e do método de validação de soluções

In [1]:
#import yaml
import copy
import numpy as np
import pandas as pd

import os
import csv
from tqdm import tqdm
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

In [2]:
from pyvrp import Model, Solution
from pyvrp.stop import MaxRuntime
from pyvrp import Client, Depot, ProblemData, VehicleType, solve as _solve
from pyvrp.plotting import plot_result

In [3]:
df=pd.read_csv("../data/niveis_inicio_fim_dez.csv", delimiter=',')

In [4]:
df.head()

,Data,Circuito,Tipo,local_id,pred_ench_3m_med,pred_ench_hgb,nivel_fim_hgb,nivel_fim_baseline
0,2024-12-02,Circ_Ilhas,Vidro,3863,2.159022,1.543212,1.543212,2.159022
1,2024-12-02,Circ_Ilhas,Vidro,3864,1.317753,1.775170,1.775170,1.317753
2,2024-12-02,Circ_Ilhas,Vidro,3844,1.135486,1.052217,1.052217,1.135486
3,2024-12-02,Circ_Ilhas,Vidro,3865,1.351893,1.089465,1.089465,1.351893
4,2024-12-02,Circ_Ilhas,Vidro,3732,1.158147,1.081761,1.081761,1.158147


In [5]:
def load_data(df):
    df['Data'] = pd.to_datetime(df['Data'])#, dayfirst=True)
    df['Data'] = df['Data'].dt.normalize()

Pré-processa o dataframe. Faz o seguinte: Converte a coluna 'Data' para datetime no formato dia/mes/ano e remove a parte da hora (dt.normalize).
Substitui os valores que são NaN nas colunas Nivel_inicio e Nivel_fim por 3. Depois para cada linha dessas colunas, pega nos valores. Se o valor da coluna Nivel_inicio for maior que o da Nivel_fim, iverte os valores.

In [6]:
CIRCUITOS = ['Circuito_01', 'Circuito_02', 'Circuito_03', 'Circuito_04',
             'Circuito_05', 'Circuito_06', 'Circuito_07', 'Circuito_08',
             'Circuito_09', 'Circuito_10', 'Circuito_11', 'Circuito_12', 'Circ_Ilhas']

In [7]:
tpath = '../sim/environments/gcvrp/sim_data/matrizes_tempo/'
dpath = '../sim/environments/gcvrp/sim_data/matrizes_distancia/'
cpath = '../sim/environments/gcvrp/sim_data/circuitos/'

mat_temp = {circ: np.loadtxt(tpath + f'matriz_tempo_{circ}.csv', delimiter=',')  for circ in CIRCUITOS} # minutes
mat_dist = {circ: np.loadtxt(dpath + f'matriz_dist_{circ}.csv', delimiter=',')  for circ in CIRCUITOS} # mete
circuitos = {circ: pd.read_csv(cpath + f'{circ}_pontos.csv')  for circ in CIRCUITOS}        

In [8]:
def construir_instance(circuito, pontos, matrizes_dist_dict, matrizes_tempo_dict, demands, capacidade):
    coords = [[p['lon'], p['lat']] for k, p in pontos.iterrows()]
    num_locs = len(coords)
    instance = {
        "coords": coords,
        "demands": demands,
        "prize": [beta * d / sum(demands) for d in demands],  # ← CORRIGIDO AQUI
        "service_time": [5] * num_locs,
        "capacity": capacidade,
        "cost_matrix": matrizes_dist_dict[circuito] / dist_scale ,
        "time_matrix": matrizes_tempo_dict[circuito],
        "start_depot_idx": 0,
        "end_depot_idx": 1
    }

    return instance

Constroi a instance com os dados 

In [9]:
def scale(data, scaling_factor):
    arr = np.array(data, dtype=float)  # converte input para array numpy float
    arr = arr * scaling_factor          # multiplica valores, não o tamanho
    arr = np.where(np.isfinite(arr), arr, 0)  # trata NaN ou inf substituindo por 0
    #arr = arr.round().astype(int)       # arredonda e converte para int
    if arr.size == 1:
        return arr.item()               # retorna escalar se só tiver um elemento
    return arr

In [10]:
def instance2data(instance: dict, scaling_factor: int, verbose=False) -> ProblemData:
    """
    Converte uma instância genérica para o formato ProblemData do PyVRP,
    preparada para problemas com clientes opcionais (PCVRP).
    """


    # Converter DataFrames para listas, se necessário
    #if isinstance(instance["cost_matrix"], pd.DataFrame):
    #    instance["cost_matrix"] = instance["cost_matrix"].values.tolist()
    #if isinstance(instance["time_matrix"], pd.DataFrame):
    #    instance["time_matrix"] = instance["time_matrix"].values.tolist()

    # Aplicar escala
    coords = scale(instance["coords"], scaling_factor)
    demands = scale(instance["demands"], scaling_factor)
    prize = scale(instance["prize"], scaling_factor)
    service = scale(instance["service_time"], scaling_factor)
    capacity = [scale(instance["capacity"], scaling_factor)]
    matrix = scale(instance["cost_matrix"], scaling_factor)
    matrix_time = scale(instance["time_matrix"], scaling_factor)

    num_locs = len(coords)
    start_depot_idx = instance.get("start_depot_idx", 0)
    end_depot_idx = instance.get("end_depot_idx", 1)

    # Verificar índices de depósitos
    if start_depot_idx < 0 or start_depot_idx >= num_locs:
        raise ValueError(f"Início do depósito inválido: {start_depot_idx}")
    if end_depot_idx < 0 or end_depot_idx >= num_locs:
        raise ValueError(f"Fim do depósito inválido: {end_depot_idx}")

    # Criar depósitos
    depots = [Depot(x=coords[start_depot_idx][0], y=coords[start_depot_idx][1])]
    if start_depot_idx != end_depot_idx:
        depots.append(Depot(x=coords[end_depot_idx][0], y=coords[end_depot_idx][1]))

    # Criar clientes opcionais (com prize)
    clients = []
    for idx in range(num_locs):
        if idx not in [start_depot_idx, end_depot_idx]:
            clients.append(Client(
                x=coords[idx][0],
                y=coords[idx][1],
                pickup=[demands[idx]],
                prize=prize[idx],
                service_duration=service[idx],
                required=False,  # ← ESSENCIAL para PCVRP
            ))

    # Tipo de veículo
    vehicle_types = [
        VehicleType(
            num_available=1,
            max_duration=420*PYVRP_SCALING_FACTOR,
            capacity=capacity,
            start_depot=start_depot_idx,
            end_depot=end_depot_idx,
        )
    ]

    # Verificação da matriz
    expected_size = num_locs
    if matrix.shape != (expected_size, expected_size):
        raise ValueError(f"Shape inconsistente: matriz {matrix.shape}, esperava {(expected_size, expected_size)}")

    if verbose:
        # Debug info
        print("→ Matrix shape:", matrix.shape)
        print("→ Num clients:", len(clients))
        print("→ Num depots:", len(depots))
        print("→ Total demand:", sum(demands))
        print("Prize: ", prize)
        print("Demands: ", demands)
        print("→ Expected matrix size:", expected_size)
        print("OPAAAA", matrix)
        print("OLAAA", matrix_time)

    return ProblemData(clients, depots, vehicle_types, [matrix], [matrix_time])

In [11]:
def solution2action(solution: Solution) -> list[int]:
    """
    Converte uma solução do PyVRP para a representação de ações (tour gigante),
    incluindo explicitamente os índices de depósito inicial e final.
    """
    action = []
    
    for route in solution.routes():
        # Adicionar o depósito inicial
        action.append(route.start_depot())
        
        # Adicionar as visitas dos clientes
        action.extend(route.visits())
        
        # Adicionar o depósito final
        action.append(route.end_depot())
        
    return action


In [12]:
def solve(instance, max_runtime, **kwargs):
    data = instance2data(instance, PYVRP_SCALING_FACTOR)
    stop = MaxRuntime(max_runtime)
    result = _solve(data, stop, **kwargs) # A função de resolução retorna um Result
    solution = result.best # A melhor solução está no atributo 'best'
    cost = result.cost() / PYVRP_SCALING_FACTOR # Assumindo que 'cost' é um atributo da Solution
    #print("→ Prize total recolhido:", solution.prize())
    #print("→ Custo (distância total):", solution.cost())
    
    return solution, cost

In [14]:
def calc_sub_rota(actions, demands_fim, matrizes_dist_dict, matrizes_tempo_dict , circuito, verbose=False):

    circuito_esc_tempo=matrizes_tempo_dict[circuito]
    circuito_esc_dist=matrizes_dist_dict[circuito]

    service_time = 5
    sub_rota = [actions[0]]  # incluir o depósito inicial
    tempo = 0
    dist = 0
    curr_load = 0
    cap = 120

    for k in range(1, len(actions)-1):
        serv_ant = actions[k - 1]
        serv = actions[k]
        #print(serv)
        demanda = demands_fim[serv]
        if verbose:
            print("O ponto é", serv)
            print("A sua demanda é", demanda)

            print(f"curr_load: {curr_load} ({type(curr_load)})")
            print(f"demanda: {demanda} ({type(demanda)})")
            print(f"capacidade: {cap} ({type(cap)})")
            print(f"soma: {curr_load + demanda}")
            print(f"Condição: {curr_load + demanda <= cap}")
        if curr_load + demanda <= cap:
            curr_load += demanda
            sub_rota.append(serv)
            tempo += circuito_esc_tempo[serv_ant, serv]
            tempo += service_time
            dist += circuito_esc_dist[serv_ant, serv]

        else:
            carga_possivel = cap - curr_load

            if carga_possivel > 0:
                #print("A carga é",carga_possivel)
                curr_load += carga_possivel
                tempo += circuito_esc_tempo[serv_ant, serv]
                tempo += service_time
                dist += circuito_esc_dist[serv_ant, serv]
                break
    # Finaliza a rota
    sub_rota.append(actions[-1])
    tempo += circuito_esc_tempo[serv, actions[-1]]
    dist += circuito_esc_dist[serv, actions[-1]]
    return sub_rota, tempo, dist, curr_load

In [15]:
PYVRP_SCALING_FACTOR = 1_000_000
beta = 10
dist_scale = 20000

In [16]:
load_data(df)

In [18]:
def run_solvers_1(data, circuito, tipo, df_filtrado):
    demands_inicio_base = np.hstack([np.zeros(2), df_filtrado['pred_ench_3m_med'].to_numpy()]) 
    demands_inicio_hgb = np.hstack([np.zeros(2), df_filtrado['pred_ench_hgb'].to_numpy()]) 

    instance_inicio_base = construir_instance(
    circuito = circuito,
    pontos = pontos_do_circuito,
    matrizes_dist_dict = mat_dist,
    matrizes_tempo_dict = mat_temp,
    demands = demands_inicio_base,
    capacidade = capacidade
    )

    instance_inicio_hgb = construir_instance(
    circuito = circuito,
    pontos = pontos_do_circuito,
    matrizes_dist_dict = mat_dist,
    matrizes_tempo_dict = mat_temp,
    demands = demands_inicio_hgb,
    capacidade = capacidade
    )

    solution_inicio_base, cost_inicio_base= solve(instance_inicio_base, max_runtime=10)
    #print("Custo base (distância):", cost_inicio_base)
    solution_inicio_hgb, cost_inicio_hgb= solve(instance_inicio_hgb, max_runtime=10)
    #print("Custo hgb (distância):", cost_inicio_hgb)

    actions_inicio_base = solution2action(solution_inicio_base)
    actions_inicio_hgb = solution2action(solution_inicio_hgb)
    
    custo_base=cost_inicio_base
    distance_base = solution_inicio_base.distance() / PYVRP_SCALING_FACTOR * 20000
    duration_base = solution_inicio_base.duration() / PYVRP_SCALING_FACTOR
    uncollected_prizes_base = solution_inicio_base.uncollected_prizes() * sum(demands_inicio_base) / (10 * PYVRP_SCALING_FACTOR)
    prizes_base = solution_inicio_base.prizes()*sum(demands_inicio_base)/ (10 * PYVRP_SCALING_FACTOR)
    num_clients_base = solution_inicio_base.num_clients()
    ocupacao_camiao_inicio_base = prizes_base

    custo_hgb=cost_inicio_base
    distance_hgb = solution_inicio_hgb.distance() / PYVRP_SCALING_FACTOR * 20000
    duration_hgb = solution_inicio_hgb.duration() / PYVRP_SCALING_FACTOR
    uncollected_prizes_hgb = solution_inicio_hgb.uncollected_prizes() * sum(demands_inicio_hgb) / (10 * PYVRP_SCALING_FACTOR)
    prizes_hgb = solution_inicio_hgb.prizes()*sum(demands_inicio_hgb)/ (10 * PYVRP_SCALING_FACTOR)
    num_clients_hgb = solution_inicio_hgb.num_clients()
    ocupacao_camiao_inicio_hgb = prizes_hgb
    

    demands_fim = np.hstack([np.zeros(2), df_filtrado['nivel_fim_hgb'].to_numpy()]) 

    instance_fim = construir_instance(
    circuito=circuito,
    pontos=pontos_do_circuito,
    matrizes_dist_dict = mat_dist,
    matrizes_tempo_dict = mat_temp,
    demands=demands_fim,
    capacidade=capacidade
    )

    sub_rota_base, tempo_base, dist_base, curr_load_base = calc_sub_rota(actions_inicio_base, demands_fim, mat_dist,mat_temp, circuito)
    sub_rota_hgb, tempo_hgb, dist_hgb, curr_load_hgb = calc_sub_rota(actions_inicio_hgb, demands_fim, mat_dist, mat_temp, circuito)

    #print("Ações", sub_rota_base)
    #print("Distancia", dist_base)
    #print("Duração", tempo_base)
    prizes_un_base = max(0, sum(demands_fim) - curr_load_base)
    #print("Prêmios não coletados", prizes_un_base)
    racio_base=prizes_un_base/sum(demands_fim) *100
    #print("Prêmios não coletados (%)", racio_base)
    #print("Prêmios coletados", curr_load_base)
    #print("Total de lixo", sum(demands_fim))

    #print("Ações", sub_rota_hgb)
    #print("Distancia", dist_hgb)
    #print("Duração", tempo_hgb)
    prizes_un_hgb = max(0, sum(demands_fim) - curr_load_hgb)
    #print("Prêmios não coletados", prizes_un_hgb)
    racio_hgb=prizes_un_hgb/sum(demands_fim) *100
    #print("Prêmios não coletados (%)", racio_hgb)
    #print("Prêmios coletados", curr_load_hgb)
    #print("Total de lixo", sum(demands_fim))

    

    

    solution_fim, cost_fim = solve(instance_fim, max_runtime=10)
    
    actions_fim = solution2action(solution_fim)
    custo_fim=cost_fim
    distance_fim = solution_fim.distance() / PYVRP_SCALING_FACTOR * 20000
    duration_fim = solution_fim.duration() / PYVRP_SCALING_FACTOR
    uncollected_prizes_fim = solution_fim.uncollected_prizes() * sum(demands_fim) / (10 * PYVRP_SCALING_FACTOR)
    prizes_fim = solution_fim.prizes()*sum(demands_fim)/ (10 * PYVRP_SCALING_FACTOR)
    num_clients_fim = solution_fim.num_clients()
    ocupacao_camiao_fim = prizes_fim

    result = {
        "data": data,
        "circuito": circuito,
        "tipo": tipo,

        # Rota Inicio Base
        "rota_inicio_base_acoes": actions_inicio_base,
        "rota_inicio_base_distancia": distance_base,
        "rota_inicio_base_duracao": duration_base,
        "rota_inicio_base_premios_nao_coletados": uncollected_prizes_base,
        "rota_inicio_base_premios_nao_coletados_pct": uncollected_prizes_base / sum(demands_inicio_base) * 100,
        "rota_inicio_base_premios_coletados": prizes_base,
        "rota_inicio_base_total_lixo": sum(demands_inicio_base),
        "rota_inicio_base_funcao_objetivo": custo_base,
        "rota_inicio_base_ocupacao_camiao": ocupacao_camiao_inicio_base,
        "rota_inicio_base_clientes_atendidos": num_clients_base,

        # Rota Inicio HGB
        "rota_inicio_hgb_acoes": actions_inicio_hgb,
        "rota_inicio_hgb_distancia": distance_hgb,
        "rota_inicio_hgb_duracao": duration_hgb,
        "rota_inicio_hgb_premios_nao_coletados": uncollected_prizes_hgb,
        "rota_inicio_hgb_premios_nao_coletados_pct": uncollected_prizes_hgb / sum(demands_inicio_hgb) * 100,
        "rota_inicio_hgb_premios_coletados": prizes_hgb,
        "rota_inicio_hgb_total_lixo": sum(demands_inicio_hgb),
        "rota_inicio_hgb_funcao_objetivo": custo_hgb,
        "rota_inicio_hgb_ocupacao_camiao": ocupacao_camiao_inicio_hgb,
        "rota_inicio_hgb_clientes_atendidos": num_clients_hgb,

        # Subrota Base
        "subrota_base_acoes": sub_rota_base,
        "subrota_base_distancia": dist_base,
        "subrota_base_duracao": tempo_base,
        "subrota_base_premios_nao_coletados": prizes_un_base,
        "subrota_base_premios_nao_coletados_pct": racio_base,
        "subrota_base_premios_coletados": curr_load_base,
        "subrota_base_total_lixo": sum(demands_fim),

        # Subrota HGB
        "subrota_hgb_acoes": sub_rota_hgb,
        "subrota_hgb_distancia": dist_hgb,
        "subrota_hgb_duracao": tempo_hgb,
        "subrota_hgb_premios_nao_coletados": prizes_un_hgb,
        "subrota_hgb_premios_nao_coletados_pct": racio_hgb,
        "subrota_hgb_premios_coletados": curr_load_hgb,
        "subrota_hgb_total_lixo": sum(demands_fim),

        # Rota Fim
        "rota_fim_acoes": actions_fim,
        "rota_fim_distancia": distance_fim,
        "rota_fim_duracao": duration_fim,
        "rota_fim_premios_nao_coletados": uncollected_prizes_fim,
        "rota_fim_premios_nao_coletados_pct": uncollected_prizes_fim / sum(demands_fim) * 100,
        "rota_fim_premios_coletados": prizes_fim,
        "rota_fim_total_lixo": sum(demands_fim),
        "rota_fim_funcao_objetivo": custo_fim,
        "rota_fim_ocupacao_camiao": ocupacao_camiao_fim,
        "rota_fim_clientes_atendidos": num_clients_fim,
    }

    return result

In [19]:
#ANO TODO

df["Data"] = pd.to_datetime(df["Data"])

combinacoes_unicas = df[["Data", "Circuito", "Tipo"]].drop_duplicates()
#combinacoes_unicas=combinacoes_unicas.head(2)

capacidade = 120

In [20]:
combinacoes_unicas

,Data,Circuito,Tipo
0,2024-12-02,Circ_Ilhas,Vidro
42,2024-12-02,Circuito_01,Papel
106,2024-12-02,Circuito_01,Vidro
170,2024-12-02,Circuito_02,Papel
240,2024-12-02,Circuito_03,Papel
...,...,...,...
28357,2024-12-31,Circuito_10,Embalagens
28433,2024-12-31,Circuito_10,Vidro
28509,2024-12-31,Circuito_11,Embalagens
28577,2024-12-31,Circuito_12,Embalagens


In [21]:
resultados = []
for index, row in tqdm(combinacoes_unicas.iterrows(), total=combinacoes_unicas.shape[0], desc="A executar solvers"):
    data = row["Data"]
    circuito = row["Circuito"]
    tipo = row["Tipo"]

    
    df_filtrado = df[
        (df['Data'].dt.date == data) &
        (df['Circuito'] == circuito) &
        (df['Tipo'] == tipo)
    ]

    pontos_do_circuito = circuitos[circuito]
    linha = run_solvers_1(data, circuito, tipo, df_filtrado)
    resultados.append(linha)

df_resultado = pd.DataFrame(resultados)
df_resultado.to_csv("../data/solucao_solvers_dezembro_final.csv", index=False)

C:\Users\simao\AppData\Local\Temp\ipykernel_20032\73563591.py:9: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  (df['Data'].dt.date == data) &
A executar solvers: 100%|██████████████████████████████████████████████████████████| 392/392 [3:18:18<00:00, 30.35s/it]
